# 第6章　PyTorchを使ったリカレントネットワーク






In [0]:
!pip install torchvision==0.2.1
!pip install pillow==4.2.0

    100% |████████████████████████████████| 61kB 2.3MB/s 
    15% |█

In [0]:
import torch
torch.__version__

'0.4.1'

# LSTMを使用した手書き文字画像分類

In [0]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

In [0]:
# データとモデルに.to(device)を指定してgpuの計算資源を使用する。
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
# ハイパーパラメータ
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

In [4]:
# MNISTデータセット
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

Processing...
Done!


In [0]:
# Recurrent neural network (多対一)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        #隠れ層（hidden_size）128ユニット
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        #入力サイズ（input_size）28、
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # 初期隠れ状態とセル状態を設定
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # LSTMを伝播する
        # outの出力形状：（バッチサイズ、シーケンス長、出力次元）
        out, _ = self.lstm(x, (h0, c0))
        
        # 最後のタイムステップの隠れ状態をデコード
        out = self.fc(out[:, -1, :])
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)


In [0]:
# 誤差関数、最適化関数を定義
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
# ネットワークを学習
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #シーケンス長（sequence_length）：28
        #入力サイズ（input_size）：28
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # 順方向に計算して誤差を算出
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 逆伝播の計算とパラメータ更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.5065
Epoch [1/2], Step [200/600], Loss: 0.1507
Epoch [1/2], Step [300/600], Loss: 0.2782
Epoch [1/2], Step [400/600], Loss: 0.1499
Epoch [1/2], Step [500/600], Loss: 0.1277
Epoch [1/2], Step [600/600], Loss: 0.2444
Epoch [2/2], Step [100/600], Loss: 0.1612
Epoch [2/2], Step [200/600], Loss: 0.0630
Epoch [2/2], Step [300/600], Loss: 0.0317
Epoch [2/2], Step [400/600], Loss: 0.0626
Epoch [2/2], Step [500/600], Loss: 0.0814
Epoch [2/2], Step [600/600], Loss: 0.0211


In [8]:
# ネットワークをテストする
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# ネットワークを保存する
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 97.71 %


## 双方向RNN

In [0]:
# Bidirectional recurrent neural network (多対一)
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        #bidirectional=Trueにして
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # LSTMを伝播する
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)

# 誤差関数、最適化関数を定義
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [10]:
# ネットワークを学習
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #シーケンス長（sequence_length）：28
        #入力サイズ（input_size）：28
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # 順方向に計算して誤差を算出
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 逆伝播の計算とパラメータ更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.5492
Epoch [1/2], Step [200/600], Loss: 0.5655
Epoch [1/2], Step [300/600], Loss: 0.0965
Epoch [1/2], Step [400/600], Loss: 0.2013
Epoch [1/2], Step [500/600], Loss: 0.0730
Epoch [1/2], Step [600/600], Loss: 0.1242
Epoch [2/2], Step [100/600], Loss: 0.0910
Epoch [2/2], Step [200/600], Loss: 0.0344
Epoch [2/2], Step [300/600], Loss: 0.0568
Epoch [2/2], Step [400/600], Loss: 0.1343
Epoch [2/2], Step [500/600], Loss: 0.0246
Epoch [2/2], Step [600/600], Loss: 0.0373


In [11]:
# ネットワークをテストする
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# ネットワークを保存する
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 97.6 %
